# Imports

In [1]:
import math
import pandas as pd
import pennylane as qml
import time

from keras.datasets import mnist
from matplotlib import pyplot as plt
from pennylane import numpy as np
from pennylane.templates import AmplitudeEmbedding, AngleEmbedding
from pennylane.templates.subroutines import ArbitraryUnitary
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Model Params

In [2]:
np.random.seed(131)
initial_params = np.random.random([165])

INITIALIZATION_METHOD = 'Angle'
BATCH_SIZE = 20
EPOCHS = 400

STEP_SIZE = 0.01
BETA_1 = 0.9
BETA_2 = 0.99
EPSILON = 0.00000001

TRAINING_SIZE = 0.78
VALIDATION_SIZE = 0.07
TEST_SIZE = 1-TRAINING_SIZE-VALIDATION_SIZE

initial_time = time.time()

# Import dataset

In [3]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
examples = np.append(train_X, test_X, axis=0)
examples = examples.reshape(70000, 28*28)
classes = np.append(train_y, test_y)

In [4]:
x = []
y = []
for (example, label) in zip(examples, classes):
    if label in [0, 2, 4, 6, 8]:
        x.append(example)
        y.append(-1)
    else:
        x.append(example)
        y.append(1)

In [5]:
x = np.array(x)
y = np.array(y)

# Normalize pixels values
x = x / 255

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, shuffle=True)

In [6]:
validation_indexes = np.random.random_integers(len(X_train), size=(math.floor(len(X_train)*VALIDATION_SIZE),))
X_validation = [X_train[n] for n in validation_indexes]
y_validation = [y_train[n] for n in validation_indexes]

In [7]:
pca = PCA(n_components=8)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_validation = pca.transform(X_validation)
X_test = pca.transform(X_test)

preprocessing_time = time.time()

# Circuit creation

In [8]:
device = qml.device("default.qubit", wires=8)

In [9]:
@qml.qnode(device)
def circuit(features, params):
    # Load state
    if INITIALIZATION_METHOD == 'Amplitude':
        AmplitudeEmbedding(features=features, wires=range(8), normalize=True, pad_with=0.)
    else:
        AngleEmbedding(features=features, wires=range(8), rotation='Y')

    # First layer
    ArbitraryUnitary(params[:15], wires=[1, 2])
    ArbitraryUnitary(params[15:30], wires=[3, 4])
    ArbitraryUnitary(params[30:45], wires=[5, 6])

    # Second layer
    ArbitraryUnitary(params[45:60], wires=[0, 1])
    ArbitraryUnitary(params[60:75], wires=[2, 3])
    ArbitraryUnitary(params[75:90], wires=[4, 5])
    ArbitraryUnitary(params[90:105], wires=[6, 7])

    # Third layer
    ArbitraryUnitary(params[105:120], wires=[2, 5])

    # Fourth layer
    ArbitraryUnitary(params[120:135], wires=[1, 2])
    ArbitraryUnitary(params[135:150], wires=[5, 6])

    # Fifth layer
    ArbitraryUnitary(params[150:165], wires=[2, 5])

    # Measurement
    return qml.expval(qml.PauliZ(5))

## Circuit example

In [10]:
features = X_train[0]
print(f"Inital parameters: {initial_params}\n")
print(f"Example features: {features}\n")
print(f"Expectation value: {circuit(features, initial_params)}\n")
print(circuit.draw())

Inital parameters: [0.65015361 0.94810917 0.38802889 0.64129616 0.69051205 0.12660931
 0.23946678 0.25415707 0.42644165 0.83900255 0.74503365 0.38067928
 0.26169292 0.05333379 0.43689638 0.20897912 0.59441102 0.09890353
 0.22409353 0.5842624  0.95908107 0.20988382 0.66133746 0.50261295
 0.32029143 0.12506485 0.80688893 0.98696002 0.54304141 0.23132314
 0.60351254 0.17669598 0.88653747 0.58902228 0.72117264 0.27567029
 0.78811469 0.1326223  0.39971595 0.62982409 0.42404345 0.16187284
 0.52034418 0.6070413  0.5808057  0.82111597 0.98499188 0.93449492
 0.90305486 0.3380262  0.78324429 0.74373474 0.58058546 0.43266356
 0.66792795 0.23668741 0.45173663 0.91999741 0.96687301 0.76905057
 0.32671177 0.62283984 0.19160224 0.24832171 0.11683869 0.01032549
 0.84869439 0.53066622 0.49383494 0.65230764 0.22667618 0.61716246
 0.2005521  0.49093199 0.94852991 0.98197027 0.17315162 0.74251972
 0.38646803 0.65110712 0.7434636  0.24777039 0.49229597 0.42474983
 0.29006936 0.98668888 0.33945016 0.1997036

# Accuracy test definition

In [11]:
def measure_accuracy(x, y, circuit_params):
    class_errors = 0

    for example, example_class in zip(x, y):
        predicted_value = circuit(example, circuit_params)

        if (example_class > 0 and predicted_value <= 0) or (example_class <= 0 and predicted_value > 0):
            class_errors += 1

    return 1 - (class_errors/len(y))

# Training

In [12]:
params = initial_params
opt = qml.AdamOptimizer(stepsize=STEP_SIZE, beta1=BETA_1, beta2=BETA_2, eps=EPSILON)
test_accuracies = []
best_validation_accuracy = 0.0
best_params = []

for i in range(len(X_train)):
    features = X_train[i]
    expected_value = y_train[i]

    def cost(circuit_params):
        value = circuit(features, circuit_params)
        return ((expected_value - value) ** 2)/len(X_train)

    params = opt.step(cost, params)

    if i % BATCH_SIZE == 0:
        print(f"epoch {i//BATCH_SIZE}")

    if i % (10*BATCH_SIZE) == 0:
        current_accuracy = measure_accuracy(X_validation, y_validation, params)
        test_accuracies.append(current_accuracy)
        print(f"accuracy: {current_accuracy}")

        if current_accuracy > best_validation_accuracy:
            print("best accuracy so far!")
            best_validation_accuracy = current_accuracy
            best_params = params

    if len(test_accuracies) == 30:
        print(f"test_accuracies: {test_accuracies}")

        if np.allclose(best_validation_accuracy, test_accuracies[0]):
            params = best_params
            break

        del test_accuracies[0]

epoch 0
accuracy: 0.45042016806722684
best accuracy so far!
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
accuracy: 0.6256902761104441
best accuracy so far!
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
accuracy: 0.6321728691476591
best accuracy so far!
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
accuracy: 0.6866746698679471
best accuracy so far!
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
accuracy: 0.685234093637455
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
accuracy: 0.6773109243697479
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
accuracy: 0.7008403361344537
best accuracy so far!
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
accuracy: 0.7070828331332533
best accuracy so far!
epoch 7

epoch 341
epoch 342
epoch 343
epoch 344
epoch 345
epoch 346
epoch 347
epoch 348
epoch 349
epoch 350
accuracy: 0.7121248499399759
test_accuracies: [0.7008403361344537, 0.7070828331332533, 0.707563025210084, 0.7207683073229292, 0.7241296518607443, 0.6669867947178871, 0.707563025210084, 0.7214885954381753, 0.7174069627851141, 0.689795918367347, 0.7236494597839136, 0.7188475390156062, 0.7226890756302521, 0.7181272509003602, 0.7164465786314526, 0.7231692677070829, 0.7183673469387755, 0.7090036014405763, 0.7114045618247299, 0.7195678271308523, 0.7241296518607443, 0.7106842737094838, 0.7320528211284514, 0.7224489795918367, 0.726530612244898, 0.7068427370948379, 0.7114045618247299, 0.7327731092436975, 0.7267707082833132, 0.7121248499399759]
epoch 351
epoch 352
epoch 353
epoch 354
epoch 355
epoch 356
epoch 357
epoch 358
epoch 359
epoch 360
accuracy: 0.7303721488595438
test_accuracies: [0.7070828331332533, 0.707563025210084, 0.7207683073229292, 0.7241296518607443, 0.6669867947178871, 0.707563025

epoch 452
epoch 453
epoch 454
epoch 455
epoch 456
epoch 457
epoch 458
epoch 459
epoch 460
accuracy: 0.7174069627851141
test_accuracies: [0.7188475390156062, 0.7226890756302521, 0.7181272509003602, 0.7164465786314526, 0.7231692677070829, 0.7183673469387755, 0.7090036014405763, 0.7114045618247299, 0.7195678271308523, 0.7241296518607443, 0.7106842737094838, 0.7320528211284514, 0.7224489795918367, 0.726530612244898, 0.7068427370948379, 0.7114045618247299, 0.7327731092436975, 0.7267707082833132, 0.7121248499399759, 0.7303721488595438, 0.702280912364946, 0.7339735894357743, 0.723889555822329, 0.7090036014405763, 0.7222088835534214, 0.7289315726290516, 0.7248499399759905, 0.7130852340936374, 0.7284513805522208, 0.7174069627851141]
epoch 461
epoch 462
epoch 463
epoch 464
epoch 465
epoch 466
epoch 467
epoch 468
epoch 469
epoch 470
accuracy: 0.7006002400960385
test_accuracies: [0.7226890756302521, 0.7181272509003602, 0.7164465786314526, 0.7231692677070829, 0.7183673469387755, 0.7090036014405763,

epoch 563
epoch 564
epoch 565
epoch 566
epoch 567
epoch 568
epoch 569
epoch 570
accuracy: 0.6969987995198079
test_accuracies: [0.7320528211284514, 0.7224489795918367, 0.726530612244898, 0.7068427370948379, 0.7114045618247299, 0.7327731092436975, 0.7267707082833132, 0.7121248499399759, 0.7303721488595438, 0.702280912364946, 0.7339735894357743, 0.723889555822329, 0.7090036014405763, 0.7222088835534214, 0.7289315726290516, 0.7248499399759905, 0.7130852340936374, 0.7284513805522208, 0.7174069627851141, 0.7006002400960385, 0.7099639855942377, 0.7164465786314526, 0.7152460984393758, 0.7260504201680672, 0.7051620648259304, 0.7212484993997599, 0.7092436974789915, 0.7133253301320528, 0.7277310924369749, 0.6969987995198079]
epoch 571
epoch 572
epoch 573
epoch 574
epoch 575
epoch 576
epoch 577
epoch 578
epoch 579
epoch 580
accuracy: 0.719327731092437
test_accuracies: [0.7224489795918367, 0.726530612244898, 0.7068427370948379, 0.7114045618247299, 0.7327731092436975, 0.7267707082833132, 0.712124849

In [13]:
print("Optimized rotation angles: {}".format(params))

training_time = time.time()

Optimized rotation angles: [ 0.5374455   1.35890894  1.10107675  1.65089988  0.86265683 -0.30249167
 -0.01947819 -0.18514236  0.28044963 -0.0642896   0.56336173  0.15257076
  1.6452681   0.07457545  0.90470188 -0.19624083  0.7526144  -0.26197089
  0.03269425  0.0073135   0.32265281  0.95334263  0.05084117  1.28849416
  0.32895215 -0.30756775  0.53089574  0.6902527  -0.05498808  0.43679152
  1.41958937 -0.21246892  0.27373853  1.57444149 -0.19401835 -0.28420741
  1.79302879 -0.10682341 -0.63626929  1.49277656  2.2153588  -0.11651053
 -0.00563931  0.44954425 -0.94164671  0.2870522   0.30931965  0.84732281
  1.36172047  0.8235738   0.5014208   0.63750553  0.26800123  1.14984689
 -0.10662478  2.15619769  0.84760303  0.5892386   1.19974197  0.99179085
  0.23141585  0.42223711  0.72720218 -0.31159543 -0.04912039  0.72118848
  0.33472966  0.64238252  0.81067499  0.96987753  0.75480944 -0.13318299
  0.16111693  0.81177831  0.94852991 -0.13049117  0.25863549  1.35744988
  0.72053292  0.82546601

# Testing

In [14]:
accuracy = measure_accuracy(X_test, y_test, params)
print(accuracy)

test_time = time.time()

0.7403809523809524


In [15]:
print(f"pre-processing time: {preprocessing_time-initial_time}")
print(f"training time: {training_time - preprocessing_time}")
print(f"test time: {test_time - training_time}")
print(f"total time: {test_time - initial_time}")

pre-processing time: 7.77047872543335
training time: 25671.73556828499
test time: 752.0479755401611
total time: 26431.554022550583
